In [1]:
import requests
import json
import time
import pandas as pd
import mysql.connector
from IPython.display import clear_output
from datetime import datetime
from config import api_football_key, conn_host, conn_database, conn_user, conn_password
import os

In [2]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [3]:
def get_winner(home_score, away_score):
    if home_score > away_score:
        return 'H'
    elif away_score > home_score:
        return 'A'
    else:
        return 'D'

In [4]:
league_id = 40 # Choose the league id
start_season = 2012
end_season = 2022

In [5]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.name AS league, ht.id as home_id, at.id as away_id, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE (l.id = {league_id} AND m.season >= {start_season} AND m.season <= {end_season}) ORDER BY m.date ASC")
fixtures_df['winner'] = fixtures_df.apply(lambda x: get_winner(x['home_score'], x['away_score']), axis=1)

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
print(len(fixtures_df.index))

5689


In [7]:
fixtures_df.head()

,id,date,season,league,home_id,away_id,home_team,away_team,home_score,away_score,home_odds,away_odds,draw_odds,winner
0,446547,2012-08-17 15:45:00,2012,Championship (England),43,37,Cardiff,Huddersfield,1,0,1.72,4.75,3.63,H
1,446548,2012-08-18 08:45:00,2012,Championship (England),63,39,Leeds,Wolves,1,0,2.22,3.11,3.37,H
2,446549,2012-08-18 11:00:00,2012,Championship (England),44,68,Burnley,Bolton,2,0,2.73,2.53,3.27,H
3,446550,2012-08-18 11:00:00,2012,Championship (England),57,67,Ipswich,Blackburn,1,1,2.45,2.79,3.30,D
4,446551,2012-08-18 11:00:00,2012,Championship (England),52,38,Crystal Palace,Watford,2,3,2.51,2.83,3.17,A


In [8]:
n_last_games = 5

In [9]:
def get_games_results(games, cenario):
    loser = 'A' if cenario == 'H' else 'H'
    return len(games.loc[games['winner'] == cenario].index), len(games.loc[games['winner'] == 'D'].index), len(games.loc[games['winner'] == loser].index)

def get_goals_mean(games, team_id, cenario):
    games = games.iloc[-n_last_games:,:]
    
    home_games = games.loc[(games['home_id'] == team_id)]
    away_games = games.loc[(games['away_id'] == team_id)]
    total_games = len(home_games.index) + len(away_games.index)
    
    home_scored_goals = home_games['home_score'].sum()
    away_scored_goals = away_games['away_score'].sum()
    total_scored_goals = home_scored_goals + away_scored_goals
    
    home_conceded_goals = home_games['away_score'].sum()
    away_condeded_goals = away_games['home_score'].sum()
    total_conceded_goals = home_conceded_goals + away_condeded_goals
    
    return_list = [total_scored_goals / total_games, total_conceded_goals / total_games]
    if cenario == 'H':
        return_list.extend([home_scored_goals / len(home_games.index), home_conceded_goals / len(home_games.index)])
    else:
        return_list.extend([away_scored_goals / len(away_games.index), away_condeded_goals / len(away_games.index)])
    
    return return_list

def get_historical_stats(home_games, away_games):
    total_games = len(home_games.index) + len(away_games.index)
    home_wins, home_draws, home_losses = get_games_results(home_games, 'H')
    away_wins, away_draws, away_losses = get_games_results(away_games, 'A')
    
    total_wins = home_wins + away_wins
    total_draws = home_draws + away_draws
    total_losses = home_losses + away_losses
    
    win_pct = total_wins * 100 / total_games
    draw_pct = total_draws * 100 / total_games
    loss_pct = total_losses * 100 / total_games
    
    points_achieved = total_wins * 3 + total_draws
    points_pct = (points_achieved * 100) / (total_games * 3)
    
    return points_pct, win_pct, draw_pct, loss_pct, home_wins, home_draws, home_losses, away_wins, away_draws, away_losses
    

def get_team_previous_games_stats(team_id, season, game_date, cenario):
    previous_games = fixtures_df.loc[((fixtures_df['home_id'] == team_id) | (fixtures_df['away_id'] == team_id)) & (fixtures_df['date'] < game_date) & (fixtures_df['season'] == season)]
    home_games = previous_games.loc[(previous_games['home_id'] == team_id)]
    away_games = previous_games.loc[(previous_games['away_id'] == team_id)]
    
    total_games = len(home_games.index) + len(away_games.index)
    if total_games < 10 or (len(home_games.index) < 5 and cenario == 'H') or (len(away_games.index) < 5 and cenario == 'A'):
        return
    
    points_pct, win_pct, draw_pct, loss_pct, home_wins, home_draws, home_losses, away_wins, away_draws, away_losses = get_historical_stats(home_games, away_games)
    
    previous_last_games = previous_games.iloc[-n_last_games:,:]
    home_last_games = previous_last_games.loc[(previous_last_games['home_id'] == team_id)]
    away_last_games = previous_last_games.loc[(previous_last_games['away_id'] == team_id)]
    
    points_pct_last_games, win_pct_last_games, draw_pct_last_games, loss_pct_last_games, home_wins_last_games, home_draws_last_games, home_losses_last_games, away_wins_last_games, away_draws_last_games, away_losses_last_games = get_historical_stats(home_last_games, away_last_games)
    
    if cenario == 'H':
        ha_win_pct = home_wins * 100 / len(home_games.index)
        ha_draw_pct = home_draws * 100 / len(home_games.index)
        ha_loss_pct = home_losses * 100 / len(home_games.index)
    else:
        ha_win_pct = away_wins * 100 / len(away_games.index)
        ha_draw_pct = away_draws * 100 / len(away_games.index)
        ha_loss_pct = away_losses * 100 / len(away_games.index)
        
    scored_mean, conceded_mean, ha_scored_mean, ha_conceded_mean = get_goals_mean(previous_games, team_id, cenario)
    
    return [points_pct, win_pct, draw_pct, loss_pct, ha_win_pct, ha_draw_pct, ha_loss_pct, scored_mean, conceded_mean, ha_scored_mean, ha_conceded_mean, win_pct_last_games, draw_pct_last_games, loss_pct_last_games]
        

In [10]:
data_model = []
for index, game in fixtures_df.iterrows():
    clear_output(wait=True)
    
    print("{}/{}".format(index, len(fixtures_df.index)))
    
    if(pd.isnull(game['home_odds'])):
        continue
    
    home_stats = get_team_previous_games_stats(game['home_id'], game['season'], game['date'], 'H')
    if not home_stats:
        continue
        
    away_stats = get_team_previous_games_stats(game['away_id'], game['season'], game['date'], 'A')
    if not away_stats:
        continue
        
    data_model.append([game['id'], game['date'], game['season'], game['home_team'], game['away_team'], game['home_odds'], game['away_odds'], game['draw_odds']] + home_stats + away_stats + [game['winner']])

5688/5689


In [11]:
columns = ['GAME_ID', 'GAME_DATE', 'SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'HOME_ODDS', 'AWAY_ODDS', 'DRAW_ODDS',
           'HOME_PTS_PCT', 'HOME_WIN_PCT', 'HOME_DRAW_PCT', 'HOME_LOSS_PCT', 'HOME_HOME_WIN_PCT', 'HOME_HOME_DRAW_PCT', 'HOME_HOME_LOSS_PCT', f'HOME_SCORED_LAST_{n_last_games}', f'HOME_CONCEDED_LAST_{n_last_games}', f'HOME_HOME_SCORED_LAST_{n_last_games}', f'HOME_HOME_CONCEDED_LAST_{n_last_games}', f'HOME_WIN_PCT_{n_last_games}', f'HOME_DRAW_PCT_{n_last_games}', f'HOME_LOSS_PCT_{n_last_games}',
           'AWAY_PTS_PCT', 'AWAY_WIN_PCT', 'AWAY_DRAW_PCT', 'AWAY_LOSS_PCT', 'AWAY_AWAY_WIN_PCT', 'AWAY_AWAY_DRAW_PCT', 'AWAY_AWAY_LOSS_PCT', f'AWAY_SCORED_LAST_{n_last_games}', f'AWAY_CONCEDED_LAST_{n_last_games}', f'AWAY_AWAY_SCORED_LAST_{n_last_games}', f'AWAY_AWAY_CONCEDED_LAST_{n_last_games}', f'AWAY_WIN_PCT_{n_last_games}', f'AWAY_DRAW_PCT_{n_last_games}', f'AWAY_LOSS_PCT_{n_last_games}',
           'OUTCOME']
data_df = pd.DataFrame(data_model, columns=columns)

In [12]:
data_df.tail()

,GAME_ID,GAME_DATE,SEASON,HOME_TEAM,AWAY_TEAM,HOME_ODDS,AWAY_ODDS,DRAW_ODDS,HOME_PTS_PCT,HOME_WIN_PCT,...,AWAY_AWAY_DRAW_PCT,AWAY_AWAY_LOSS_PCT,AWAY_SCORED_LAST_5,AWAY_CONCEDED_LAST_5,AWAY_AWAY_SCORED_LAST_5,AWAY_AWAY_CONCEDED_LAST_5,AWAY_WIN_PCT_5,AWAY_DRAW_PCT_5,AWAY_LOSS_PCT_5,OUTCOME
4294,717343,2022-05-07 08:30:00,2021,Preston,Middlesbrough,4.49,1.76,3.91,45.185185,33.333333,...,36.363636,36.363636,1.2,0.8,0.500000,0.500000,40.0,40.0,20.0,H
4295,717344,2022-05-07 08:30:00,2021,Sheffield Utd,Fulham,2.09,3.44,3.63,53.333333,44.444444,...,22.727273,18.181818,2.4,0.8,1.000000,1.500000,40.0,20.0,40.0,H
4296,717345,2022-05-07 08:30:00,2021,Stoke,Coventry,2.19,3.17,3.67,45.185185,37.777778,...,31.818182,36.363636,1.6,1.6,2.333333,1.000000,40.0,20.0,40.0,D
4297,717346,2022-05-07 08:30:00,2021,Swansea,QPR,1.96,3.62,3.88,45.185185,35.555556,...,13.636364,50.000000,1.0,1.6,1.000000,1.666667,20.0,20.0,60.0,A
4298,717347,2022-05-07 08:30:00,2021,West Brom,Barnsley,1.26,11.33,6.19,47.407407,37.777778,...,22.727273,72.727273,0.6,2.0,1.000000,1.500000,0.0,20.0,80.0,H


In [13]:
def parse_df_to_csv(dataframe, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    dataframe.to_csv("{}/{}".format(path, filename))

In [14]:
parse_df_to_csv(data_df, f'leagues/{league_id}/data', '{}-{}.csv'.format(start_season, end_season))